# Intelligent Agents: Reflex Agents for the Vacuum-cleaner World


## Instructions

Total Points: undergrad 10, graduate students 11

Complete this notebook and submit it. The notebook needs to be a complete project report with 

* your implementation,
* documentation including a short discussion of how your implementation works and your design choices, and
* experimental results (e.g., tables and charts with simulation results) with a short discussion of what they mean. 

Use the provided notebook cells and insert additional code and markdown cells as needed.

## Introduction

In this assignment you will implement a simulator environment for an automatic vacuum cleaner robot, a set of different reflex agent programs, and perform a comparison study for cleaning a single room. Focus on the __cleaning phase__ which starts when the robot is activated and ends when the last dirty square is cleaned. Someone else will take care of the agent program needed to navigate back to the charging station after the room is clean.

## PEAS description of the cleaning phase

__Performance Measure:__ Each action costs 1 energy unit. The performance is measured as the sum of the energy units used to clean the whole room.

__Environment:__ A room with $n \times n$ squares where $n = 5$. Dirt is randomly placed on each square with probability $p = 0.2$. For simplicity, you can assume that the agent knows the size and the layout of the room (i.e., it knows $n$). To start, the agent is placed on a random square.

__Actuators:__ The agent can `clean` the current square or move to an adjacent square by going `north`, `east`, `south`, or `west`.

__Sensors:__ Four bumper sensors, one for north, east, south, and west; a dirt sensor reporting dirt in the current square.  


## The agent program for a simple randomized agent

The agent program is a function that gets sensor information (the current percepts) as the arguments. The arguments are:

* A dictionary with boolean entries for the for bumper sensors `north`, `east`, `west`, `south`. E.g., if the agent is on the north-west corner, `bumpers` will be `{"north" : True, "east" : False, "south" : False, "west" : True}`.
* The dirt sensor produces a boolean.

The agent returns the chosen action as a string.

Here is an example implementation for the agent program of a simple randomized agent:  

In [1]:
import numpy as np
import pandas as pd

np.random.seed(seed=1) # set random seed
actions = ["north", "east", "west", "south", "suck"]

def simple_randomized_agent(bumpers, dirty):
    return np.random.choice(actions)

In [2]:
# define percepts (current location is NW corner and it is dirty)
bumpers = {"north" : True, "east" : False, "south" : False, "west" : True}
dirty = True

# call agent program function with percepts and it returns an action
simple_randomized_agent(bumpers, dirty)

'south'

__Note:__ This is not a rational intelligent agent. It ignores its sensors and may bump into a wall or not clean a dirty square. You will be asked to implement rational agents below.

## Simple environment example

This simple environment is infinite in size (bumpers are always `False`) and every square is always dirty, even if the agent cleans it. The environment function returns the performance measure which is here the number of cleaned squares (since all squares are constantly dirty, it is the number of `suck` actions by the agent). 

In [3]:
def simple_environment(agent, max_steps, verbose = True):
    num_cleaned = 0
    
    for i in range(max_steps):
        dirty = True
        bumpers = {"north" : False, "south" : False, "west" : False, "east" : False}

        action = agent(bumpers, dirty)
        if (verbose): print("step", i , "- action:", action) 
        
        if (action == "suck"): 
            num_cleaned = num_cleaned + 1
        
    return num_cleaned

Do one simulation run with 20 steps.

In [4]:
simple_environment(simple_randomized_agent, max_steps = 20)

step 0 - action: suck
step 1 - action: north
step 2 - action: east
step 3 - action: south
step 4 - action: north
step 5 - action: north
step 6 - action: east
step 7 - action: suck
step 8 - action: suck
step 9 - action: east
step 10 - action: west
step 11 - action: suck
step 12 - action: west
step 13 - action: suck
step 14 - action: south
step 15 - action: suck
step 16 - action: west
step 17 - action: suck
step 18 - action: west
step 19 - action: suck


8

# Tasks

_Submission Instructions:_ Use this notebook to prepare your submission. Complete this section with your code and results. You can add additional Markdown blocks for your description, comments in the code and use mathplotlib to produce charts. 

_Note:_ Try to keep the code simple! In this course, we want to learn about the algorithms and we often do not need to use object-oriented design. 


## Task 1: Implement a simulation environment [2 Points]

The simple environment above is not very realistic. Your environment simulator needs to follow the PEAS description from above. It needs to:

* Initialize the environment by storing the state of each square (clean/dirty) and making some dirty.
* Keep track of the agent's position.
* Call the agent function repeatedly and provide the agent function with the sensor inputs.  
* React to the agent's actions. E.g, by removing dirt from a square or moving the agent around unless there is a wall in the way.
* Keep track of the performance measure. That is, track the agent's actions until all dirty squares are clean and count the number of actions it takes the agent to complete the task.

The easiest implementation for the environment is to hold an 2-dimensional array to represent if squares are clean or dirty and to call the agent function in a loop until all squares are clean or a predefined number of steps have been reached (i.e., the robot runs out of energy).

The simulation environment needs to work with the simple randomized agent program from above and then it can be used for your agent implementation in the tasks below.

In [5]:
# Description:
#   My environment creates random dirty squares and stores them in a 2D array.
#   It also tracks where the agent is and removes dirt from 2D array when appropriate.
#   In this array, element zero represents the negative vertical axis, visually, and
#   element one represents the positive horizontal axis. This is to simplify two
#   processes: printing the array and mentally converting the visual positions
#   to coordinates.

# Code:
def my_environment(agent, n = 5, max_steps = 20, verbose = True):
    
    # Initializing the environment.
    data = {'num_cleaned': 0, 'energy_used': 0}
    pos = np.random.randint(n, size=(2)) # Agent starts in a random square.
    if (verbose): print(pos)
    dirty = np.random.choice([0,1], p=[0.8, 0.2], size=(n, n))
    if (verbose): print(dirty)
    
    # Limited by battery.
    for i in range(max_steps):
        data['energy_used']+=1
        
        # Update the bumper sensors.
        bumpers = {"north" : False, "south" : False, "west" : False, "east" : False}
        if (pos[0] == 0): bumpers['north'] = True
        if (pos[0] == 4): bumpers['south'] = True
        if (pos[1] == 0): bumpers['west'] = True
        if (pos[1] == 4): bumpers['east'] = True
        
        # Send sensor info to the agent.
        action = agent(bumpers, dirty[pos[0],pos[1]])
        
        # React to the agent's action.
        if (verbose): print("step", i, "- action:", action) 
        if (action == "north" and pos[0] != 0): pos[0]-=1 # Agent can't go through walls
        elif (action == "south" and pos[0] != 4): pos[0]+=1
        elif (action == "west" and pos[1] != 0): pos[1]-=1
        elif (action == "east" and pos[1] != 4): pos[1]+=1
        elif (action == "suck" and dirty[pos[0],pos[1]]):
            # Remove dirt from 2d array, and record a cleaned square.
            dirty[pos[0],pos[1]]=0
            data['num_cleaned']+=1
            # If all clean, return data.
            if (data['num_cleaned'] == 25):
                break
                
    print(pos)
    print(dirty)
    return pd.DataFrame(data, index = [''])

In [6]:
my_environment(simple_randomized_agent)

[1 1]
[[0 0 1 1 0]
 [0 1 1 0 0]
 [0 1 0 0 1]
 [0 0 0 0 1]
 [0 0 1 0 0]]
step 0 - action: west
step 1 - action: east
step 2 - action: north
step 3 - action: east
step 4 - action: south
step 5 - action: suck
step 6 - action: south
step 7 - action: east
step 8 - action: south
step 9 - action: north
step 10 - action: north
step 11 - action: west
step 12 - action: west
step 13 - action: east
step 14 - action: south
step 15 - action: suck
step 16 - action: west
step 17 - action: north
step 18 - action: north
step 19 - action: east
[0 2]
[[0 0 1 1 0]
 [0 1 0 0 0]
 [0 1 0 0 1]
 [0 0 0 0 1]
 [0 0 1 0 0]]


,num_cleaned,energy_used
,1,20


## Task 2:  Implement a simple reflex agent [1 Point] 

The simple reflex agent randomly walks around but reacts to the bumper sensor by not bumping into the wall and to dirt with sucking. Implement the agent program as a function.

_Note:_ The agent cannot directly use variable in the environment. It only gets the percepts as the arguments to the agent program function.

In [7]:
# Description:
#   My simple reflex agent reacts to dirt by sucking, and it travels a random direction
#   where there is no wall in its way.

# Code:
travel = ["north", "east", "west", "south"]

def simple_reflex_agent(bumpers, dirty):
    if (dirty): return "suck"
    action = np.random.choice(travel)
    while (bumpers[action]): # Check for wall
        action = np.random.choice(travel)
    return action

In [8]:
my_environment(simple_reflex_agent)

[1 3]
[[0 0 0 0 0]
 [0 1 0 0 1]
 [0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 1 0]]
step 0 - action: south
step 1 - action: east
step 2 - action: west
step 3 - action: north
step 4 - action: north
step 5 - action: south
step 6 - action: west
step 7 - action: west
step 8 - action: suck
step 9 - action: west
step 10 - action: north
step 11 - action: east
step 12 - action: west
step 13 - action: south
step 14 - action: south
step 15 - action: north
step 16 - action: south
step 17 - action: east
step 18 - action: east
step 19 - action: west
[2 1]
[[0 0 0 0 0]
 [0 0 0 0 1]
 [0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 1 0]]


,num_cleaned,energy_used
,1,20


## Task 3: Implement a model-based reflex agent [3 Point]

This agent keeps track of the location and remembers where it has cleaned. Assume the agent knows how many squares the room has. It can move to a corner to determine its location and then is able to use more advanced navigation.

Describe how you define the __agent state__ and how your agent works before implementing it. _Note on implementing the state in Python:_ [Examples](https://github.com/mhahsler/CS7320-AI/blob/master/Python_Code_Examples/store_agent_state_information.ipynb)

In [9]:
# State:
#   I define agent state with a name to identify it and an array for its current coordinate position
#   after it has navigated to the northwest corner of the environment.

# Implementation:
#     My model-based reflex agent will first go west until it detects a wall west of it, then it will
#   go north until it detects another wall north of it. After this, the agent should be in the northwest
#   corner of the environment, and it will suck if dirt is detected.
#     Next, it moves east and sucks again until it detects an eastward wall. (Throughout, it will remember
#   where it has taken a DSR, dirt sensor reading, by marking that location in its state.) To continue, the
#   agent moves south to the next y-coordinate and sucks if true DSR and repeats moving west
#   until it detects a westward wall. This process engages in this snake-like traveling pattern until
#   the environment realizes all the squares are clean or the battery is out.

In [10]:
class MBR_Agent:
    def __init__(self, name = "A Model-Based Reflex Agent", nav = False):
        self.name = name
        self.nav = nav # Can the agent use advanced navigation yet?
        self.position = np.array([0,0])
        if (!nav): self.code = 0
        else: self.code = 2

    def act(self, bumpers, dirty):
        # 0. if no west wall, go west
        # 1. if no north wall, go north
        self.nav = True # Agent knows where it is now.
        # 2. mark and take DSR
        
        # until east wall
        #   3. go east
        #   4. mark and take DSR
        # 5. go south
        # 6. mark and take DSR
        # until west wall
        #   7. go west
        #   8. mark and take DSR
        
        return("error")

In [11]:
mbr_agent = MBR_Agent(initial_position = init_pos)
my_environment(mbr_agent.act)

[4 0]
[[0 0 0 0 0]
 [0 1 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]]
A Model-Based Reflex Agent
step 0 - action: suck
A Model-Based Reflex Agent
step 1 - action: suck
A Model-Based Reflex Agent
step 2 - action: suck
A Model-Based Reflex Agent
step 3 - action: suck
A Model-Based Reflex Agent
step 4 - action: suck
A Model-Based Reflex Agent
step 5 - action: suck
A Model-Based Reflex Agent
step 6 - action: suck
A Model-Based Reflex Agent
step 7 - action: suck
A Model-Based Reflex Agent
step 8 - action: suck
A Model-Based Reflex Agent
step 9 - action: suck
A Model-Based Reflex Agent
step 10 - action: suck
A Model-Based Reflex Agent
step 11 - action: suck
A Model-Based Reflex Agent
step 12 - action: suck
A Model-Based Reflex Agent
step 13 - action: suck
A Model-Based Reflex Agent
step 14 - action: suck
A Model-Based Reflex Agent
step 15 - action: suck
A Model-Based Reflex Agent
step 16 - action: suck
A Model-Based Reflex Agent
step 17 - action: suck
A Model-Based Reflex Agent
step 18 - a

,num_cleaned,energy_used
,0,20


## Task 4: Simulation study [3 Points]

Compare the performance (the performance measure is defined in the PEAS description above) of the agents using  environments of different size. E.g., $5 \times 5$, $10 \times 10$ and
$100 \times 100$. Use 100 random runs for each. Present the results in a suitable format (tables, graphs) and discuss the differences. 

Here is some help with [charts and tables.](https://github.com/mhahsler/CS7320-AI/blob/master/Python_Code_Examples/charts_and_tables.ipynb)

In [12]:
# Your code goes here

Fill out the following table with the average performance measure for 100 random runs (you may also create this table with code):

| Size     | Randomized Agent | Simple Reflex Agent | Model-based Reflex Agent |
|----------|------------------|---------------------|--------------------------|
| 5x5     | | | |
| 10x10   | | | |
| 100x100 | | | |

In [13]:
# Your discussion of the results goes here

## Task 5: Robustness of the agent implementations [1 Point] 

Describe how your agent implementations will perform 

* if it is put into a rectangular room with unknown size, 
* if the cleaning area can have an irregular shape (e.g., a hallway connecting two rooms), or 
* if the room contains obstacles (i.e., squares that it cannot pass through and trigger the bumper sensors).

In [14]:
# Answer goes here

## Graduate student advanced task: Obstacles [1 Point]

__Undergraduate students:__ This is a bonus task you can attempt if you like [+1 Bonus point].

1. Change your simulation environment tor run experiments for the following problem: Add random obstacle squares that also trigger the bumper sensor. The agent does not know where the obstacles are. Observe how this changes the performance of the three implementations.

2. Describe what would need to be done to perform better with obstacles. Add code if you can. 

In [15]:
# Your code and discussion goes here

## More advanced tasks to think about

You can think about these:

* __Unknown environment with obstacles:__ Implement an agent for an environment where the agent does not know how large the environment is (we assume it is rectangular), where it starts or where the obstacles are. An option would be to always move to the closest unchecked/uncleaned square.

* __Utility-based agent:__ Change the environment, so each square has a fixed probability of getting dirty again. We assume the agent has learned this information over time. For the implementation, we give this information to the agent as a 2-dimensional array of probabilities  Cleaning one dirty square produces a utility of 1. Implement a utility-based agent that maximizes the expected utility over one full charge which lasts for 10000 time steps. This is very tricky!

In [16]:
# Your ideas/code